In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.05_tbl_account_prism_sub_trt_prm AS
INSERT OVERWRITE TABLE teamconnect.pca_prism.05_tbl_account_prism_sub_trt_prm
SELECT
  /*+ BROADCAST(trt), BROADCAST(pa), BROADCAST(pl12) */
  b.btn,
  b.dpi_environment,

  -- Subscriber
  COALESCE(si.customer_name,      'Unknown')               AS customer_name,
  COALESCE(si.service_address,    'Address Not Available') AS service_address,
  si.connect_date,
  si.disconnect_date,
  si.days_in_service,

  -- Treatment status
  trt.opnsps  AS suspend_open_flag,
  trt.spsdte  AS suspend_date,
  trt.opndpa  AS dpa_open_flag,
  trt.dpadte  AS dpa_date,
  trt.opnnsf  AS nsf_open_flag,
  trt.nsfdte  AS nsf_date,
  trt.opnovr  AS ovr_open_flag,
  trt.ovrdte  AS ovr_date,
  trt.remdte  AS last_reminder_date,
  trt.remamt  AS last_reminder_amount_due,
  trt.spsamt  AS last_suspended_amount,
  trt.disqualified_upsell_sne_yn,

  -- Promotions
  COALESCE(pa.active_promos,       ARRAY()) AS active_promos,
  COALESCE(pl12.recent_promos_12m, ARRAY()) AS recent_promos_12m

FROM teamconnect.pca_prism.04_tbl_base_btns b
LEFT JOIN teamconnect.pca_prism.10_tbl_subinf_cur   si
       ON si.btn = b.btn AND si.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.07_tbl_trt_latest   trt
       ON trt.btn = b.btn AND trt.dpi_environment = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.09_tbl_prm_active   pa
       ON pa.btn  = b.btn AND pa.dpi_environment  = b.dpi_environment
LEFT JOIN teamconnect.pca_prism.tbl_prm_last12   pl12
       ON pl12.btn= b.btn AND pl12.dpi_environment= b.dpi_environment;

OPTIMIZE teamconnect.pca_prism.05_tbl_account_prism_sub_trt_prm
ZORDER BY (btn, dpi_environment);
